In [85]:
import pandas as pd
import numpy as np

df_knn = pd.read_csv('KNN_resultados.csv')
df_xgb = pd.read_csv('XGBoost_resultados.csv')

In [86]:
df_knn = df_knn.drop(columns=["Unnamed: 0", "Overfitting?", "Underfitting?"])
df_knn = df_knn.rename(columns={
    'Real':'Valores Reais',
    'Previsto':'knn_Previsto',
    'Erro Absoluto':'knn_MAE',
    'Train RMSE':'knn_train_RMSE',
    'Test RMSE':'knn_test_RMSE'
        })

In [87]:
df_xgb = df_xgb.drop(columns=["Unnamed: 0", "Overfitting?", "Underfitting?"])
df_xgb = df_xgb.rename(columns={
    'Real':'Valores Reais',
    'Previsto':'xgb_Previsto',
    'Erro Absoluto':'xgb_MAE',
    'Train RMSE':'xgb_train_RMSE',
    'Test RMSE':'xgb_test_RMSE'
        })

In [88]:
df = pd.merge(df_knn, df_xgb, on=['Empresa', 'Data', 'Valores Reais'], how='right')
df

,Empresa,Data,Valores Reais,knn_Previsto,knn_MAE,knn_train_RMSE,knn_test_RMSE,xgb_Previsto,xgb_MAE,xgb_train_RMSE,xgb_test_RMSE
0,3tentos,2022-10-01,0.000718,0.009481,0.008763,0.009585,0.008763,0.009481,0.008763,0.009585,0.008763
1,3tentos,2023-01-01,-0.002094,0.005192,0.007286,0.008431,0.007286,0.007728,0.009823,0.009262,0.009823
2,3tentos,2023-04-01,-0.001256,0.000200,0.001456,0.006123,0.001456,0.006091,0.007348,0.009213,0.007348
3,3tentos,2023-07-01,0.003401,0.001250,0.002151,0.005602,0.002151,0.005042,0.001641,0.008909,0.001641
4,3tentos,2023-10-01,-0.004247,0.000192,0.004440,0.005181,0.004440,0.004837,0.009084,0.008351,0.009084
...,...,...,...,...,...,...,...,...,...,...,...
7921,Zamp S.A.,2023-07-01,0.002731,-0.000838,0.003569,0.003610,0.003569,0.001208,0.001523,0.006751,0.001523
7922,Zamp S.A.,2023-10-01,0.006019,-0.000071,0.006090,0.003572,0.006090,0.001271,0.004748,0.006616,0.004748
7923,Zamp S.A.,2024-01-01,-0.018266,-0.005160,0.013106,0.003596,0.013106,0.001461,0.019727,0.006549,0.019727
7924,Zamp S.A.,2024-04-01,0.013895,0.005300,0.008595,0.003744,0.008595,0.000702,0.013193,0.007458,0.013193


In [89]:
from sklearn.metrics import mean_squared_error
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Função para calcular RMSE
def calcular_rmse_knn(df):
    return np.sqrt(mean_squared_error(df['Valores Reais'], df['knn_Previsto']))

# Aplicar por grupo (por empresa)
rmse_por_empresa_knn = df.groupby('Empresa').apply(calcular_rmse_knn).reset_index()
rmse_por_empresa_knn.columns = ['Empresa', 'RMSE_knn']

In [90]:
# Função para calcular RMSE
def calcular_rmse_xgb(df):
    return np.sqrt(mean_squared_error(df['Valores Reais'], df['xgb_Previsto']))

# Aplicar por grupo (por empresa)
rmse_por_empresa_xgb = df.groupby('Empresa').apply(calcular_rmse_xgb).reset_index()
rmse_por_empresa_xgb.columns = ['Empresa', 'RMSE_xgb']

In [91]:
rmse_final = pd.merge(rmse_por_empresa_knn, rmse_por_empresa_xgb, on=['Empresa'], how='right')

empresa_maior_rmse_knn = rmse_final.loc[rmse_final['RMSE_knn'].idxmax()]
empresa_menor_rmse_knn = rmse_final.loc[rmse_final['RMSE_knn'].idxmin()]

# Maior e menor RMSE para o XGB
empresa_maior_rmse_xgb = rmse_final.loc[rmse_final['RMSE_xgb'].idxmax()]
empresa_menor_rmse_xgb = rmse_final.loc[rmse_final['RMSE_xgb'].idxmin()]

print("Maior RMSE KNN:", empresa_maior_rmse_knn)
print("Menor RMSE KNN:", empresa_menor_rmse_knn)

print("Maior RMSE XGB:", empresa_maior_rmse_xgb)
print("Menor RMSE XGB:", empresa_menor_rmse_xgb)

Maior RMSE KNN: Empresa     Petrobras
RMSE_knn     0.442604
RMSE_xgb     0.324413
Name: 193, dtype: object
Menor RMSE KNN: Empresa     Dtcom Direct
RMSE_knn        0.000028
RMSE_xgb        0.000027
Name: 78, dtype: object
Maior RMSE XGB: Empresa     Petrobras
RMSE_knn     0.442604
RMSE_xgb     0.324413
Name: 193, dtype: object
Menor RMSE XGB: Empresa     Dtcom Direct
RMSE_knn        0.000028
RMSE_xgb        0.000027
Name: 78, dtype: object


In [92]:
grupo = df.groupby('Empresa')[['knn_MAE', 'knn_train_RMSE', 'knn_test_RMSE', 'xgb_MAE', 'xgb_train_RMSE', 'xgb_test_RMSE']].mean().reset_index()
grupo #media do MAE, media do RMSE de treino e media do RMSE teste por modelo

,Empresa,knn_MAE,knn_train_RMSE,knn_test_RMSE,xgb_MAE,xgb_train_RMSE,xgb_test_RMSE
0,3tentos,0.005229,0.005851,0.005229,0.006244,0.008663,0.006244
1,Aco Altona,0.000354,0.000281,0.000354,0.000361,0.000370,0.000361
2,Aeris,0.003211,0.004726,0.003211,0.004010,0.006807,0.004010
3,Afluente T,0.000388,0.000294,0.000388,0.000364,0.000344,0.000364
4,Agrogalaxy,0.008917,0.007636,0.008917,0.012790,0.011182,0.012790
...,...,...,...,...,...,...,...
275,Wilson Sons,0.001594,0.001086,0.001594,0.002507,0.001897,0.002507
276,Wiz Co,0.006316,0.004089,0.006316,0.006547,0.006479,0.006547
277,Wlm Ind Com,0.000201,0.000173,0.000201,0.000181,0.000202,0.000181
278,Yduqs Part,0.005485,0.005962,0.005485,0.008619,0.011299,0.008619


In [93]:
knn_MAE_media = df['knn_MAE'].mean()
knn_train_RMSE_media = df['knn_train_RMSE'].mean()
knn_test_RMSE_media = grupo['knn_test_RMSE'].mean()

xgb_MAE_media = df['xgb_MAE'].mean()
xgb_train_RMSE_media = df['xgb_train_RMSE'].mean()
xgb_test_RMSE_media = df['xgb_test_RMSE'].mean()

print(f'MAE Médio KNN: {knn_MAE_media}') #é diferente do MAE geral do modelo pois as médias de cada empresas possuem ns diferentes (datasets com observações diferentes)
print(f'RMSE Treino Médio KNN: {knn_train_RMSE_media}')
print(f'RMSE Teste Médio KNN: {knn_test_RMSE_media}')

print(f'MAE Médio XGB: {xgb_MAE_media}')
print(f'RMSE Treino Médio XGB: {xgb_train_RMSE_media}')
print(f'RMSE Teste Médio XGB: {xgb_test_RMSE_media}')

MAE Médio KNN: 0.009458392368906266
RMSE Treino Médio KNN: 0.00993908888274029
RMSE Teste Médio KNN: 0.008573297003121443
MAE Médio XGB: 0.013974778733443025
RMSE Treino Médio XGB: 0.014617835929099474
RMSE Teste Médio XGB: 0.013974778733443025


In [94]:
knn_MAE_std = df['knn_MAE'].std()
knn_train_RMSE_std = df['knn_train_RMSE'].std()
knn_test_RMSE_std = grupo['knn_test_RMSE'].std()

xgb_MAE_std = df['xgb_MAE'].std()
xgb_train_RMSE_std = df['xgb_train_RMSE'].std()
xgb_test_RMSE_std = df['xgb_test_RMSE'].std()

print(f'MAE Std KNN: {knn_MAE_std}') #é diferente do MAE geral do modelo pois as médias de cada empresas possuem ns diferentes (datasets com observações diferentes)
print(f'RMSE Treino Std KNN: {knn_train_RMSE_std}')
print(f'RMSE Teste Std KNN: {knn_test_RMSE_std}')

print(f'MAE Std XGB: {xgb_MAE_std}')
print(f'RMSE Treino Std XGB: {xgb_train_RMSE_std}')
print(f'RMSE Teste Std XGB: {xgb_test_RMSE_std}')

MAE Std KNN: 0.0398768713513208
RMSE Treino Std KNN: 0.04726022671238258
RMSE Teste Std KNN: 0.019966259284070597
MAE Std XGB: 0.04114984947850017
RMSE Treino Std XGB: 0.03042833801038499
RMSE Teste Std XGB: 0.04114984947850017


In [95]:
from scipy.stats import friedmanchisquare
from scipy.stats import rankdata

friedman = friedmanchisquare(grupo['knn_MAE'], grupo['xgb_MAE']) #faltando 1 modelo

friedman

ValueError: At least 3 sets of samples must be given for Friedman test, got 2.

In [96]:
# Aplicar ranking por linha (menor MAE = melhor)
ranks = grupo[['knn_MAE', 'xgb_MAE']].apply(rankdata, axis=1)
mean_ranks = ranks.mean(axis=0)


# Exibir ranking médio
print("Rank médio dos modelos:")
for i, rank in enumerate(mean_ranks, start=1):
    print(f"Modelo {i}: {rank:.3f}")

Rank médio dos modelos:
Modelo 1: 1.164
Modelo 2: 1.836
